In [39]:
#!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2396804 entries, 0 to 2396803
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user_id            int64  
 1   basket_id          int64  
 2   day                int64  
 3   item_id            int64  
 4   quantity           int64  
 5   sales_value        float64
 6   store_id           int64  
 7   retail_disc        float64
 8   trans_time         int64  
 9   week_no            int64  
 10  coupon_disc        float64
 11  coupon_match_disc  float64
dtypes: float64(4), int64(8)
memory usage: 219.4 MB


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
users, items, interactions = data.user_id.nunique(), data.item_id.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу


In [9]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные взвешенные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
   
    items_weights = np.array(items_weights['weight'])
    
    recs = np.random.choice(items, n, replace=False, p = items_weights)
    
    
    return recs.tolist()

In [10]:
%%time

#сэмплируем товары, учитывая вес по популярности
popular = data.groupby('item_id')['user_id'].sum().reset_index()
popular.sort_values('user_id', ascending=False, inplace=True)
popular['prob'] = popular['user_id']/popular['user_id'].sum().round(2)
items_weights = popular[['item_id', 'prob']].rename(columns={'item_id': 'item_id', 'prob': 'weight'})

CPU times: user 78.2 ms, sys: 4.26 ms, total: 82.4 ms
Wall time: 81.3 ms


In [387]:
result = pd.read_csv('predictions_basic.csv')

In [388]:
result['actual'] = result['actual'].apply(lambda x: [int(el) for el in x.replace('[', '').replace(']', '').split()])

In [394]:
result.columns

Index(['user_id', 'actual', 'random_recommendation', 'popular_recommendation',
       'itemitem', 'cosine', 'tfidf', 'own_purchases'],
      dtype='object')

In [395]:
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]","[5592734, 831937, 883404, 871611, 1068855]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]","[5995312, 971165, 1076056, 928146, 1003188]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [396]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [397]:
# your_code

alg_columns_list = list(result.drop(['user_id', 'actual', 'weighted_random_recommendation'], axis = 1).columns)
alg_columns_list

['random_recommendation',
 'popular_recommendation',
 'itemitem',
 'cosine',
 'tfidf',
 'own_purchases']

In [398]:
el_list = [['weighted_random_recommendation', result.apply(lambda row: precision_at_k(row['weighted_random_recommendation'], row['actual']), axis=1).mean()]]

In [399]:
for el in alg_columns_list:
    el_list.append([el, result.apply(lambda row: precision_at_k(eval(row[el]), row['actual']), axis=1).mean()])

In [400]:
el_list

[['weighted_random_recommendation', 0.019000979431929486],
 ['random_recommendation', 0.0005876591576885406],
 ['popular_recommendation', 0.15523996082272082],
 ['itemitem', 0.033594515181195064],
 ['cosine', 0.03525954946131262],
 ['tfidf', 0.03614103819784544],
 ['own_purchases', 0.1799869409076044]]

In [401]:
precision_at_5 = pd.DataFrame(np.array(el_list), columns = ['alg', 'precision@5'])

In [402]:
precision_at_5

,alg,precision@5
0,weighted_random_recommendation,0.019000979431929486
1,random_recommendation,0.0005876591576885406
2,popular_recommendation,0.15523996082272082
3,itemitem,0.033594515181195064
4,cosine,0.03525954946131262
5,tfidf,0.03614103819784544
6,own_purchases,0.1799869409076044


Вывод: лучшими алгоритмами по precision@5 являются popular_recommendation и own_purchases. Алгоритм weighted_random_recommendation работает лучше, чем random_recommendation. Случайный выбор товаров с учетом их веса по популярности улучшил случайный механизм. Если использовать механизм популярных товаров, что нужно понимать, что пользователь возможно и так купит популярные товары, без нашей рекомендации, соответственно необходимо оценить целесообразность осуществления затрат на эту рекомендацию. Точно так же, сомнительным кажется использовать механизм рекомендаций для товаров, которые пользователь и так регулярно покупает, во всяком случае следует оценить финансовую эффективность таких рекомендательных систем.